In [1]:
%run -n main.py
load_dotenv(".envrc", override=True)
assert "OPENROUTER_API_KEY" in os.environ
assert "OPENAI_API_KEY" in os.environ

/Users/alexkuk/proj/wildchat-hard-ru/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run -n main.py
api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_client = OpenrouterClient(api_key=api_key)

logger.info("Update model pricing")
await openrouter_client.update_model_pricing()

logger.info("Update rate limit")
await openrouter_client.update_rate_limit()
logger.info("Rate limit reqs / sec = %d / %d", openrouter_client.rate_limiter.max_requests, openrouter_client.rate_limiter.time_window)

2025-04-28 15:30:21 Update model pricing
2025-04-28 15:30:22 Update rate limit
2025-04-28 15:30:23 Rate limit reqs / sec = 590 / 10


In [4]:
%run -n main.py
api_key = os.getenv("OPENAI_API_KEY")
openai_emb_client = OpenaiEmbClient(api_key)
response = await openai_emb_client.embeddings(["xxx"], input_tokens=1)
response["data"][0]["embedding"][:100]

'zu3YPGMD4TtbUNm8NixnPfBl4TwIGQA8pkwKvRSSNj0N10s81TVmPWkK/7yrClY8BtiQvQkRHb2/0Bs9Net3vLwCirx/nla8xI7n'

In [5]:
%run -n main.py
output_notebook()

Loading BokehJS ...

# data lmsys

In [6]:
lmsys_items = []
paths = sorted(Path('data/lmsys/').glob('*.parquet'))
for path in paths:
    df = pd.read_parquet(
        path=path,
        filters=[('language', '==', 'Russian')]
    )
    logger.info('path="%s" len(df)=%d', path, len(df))
    for item in df.to_dict(orient="records"):
        if item["redacted"]:
            continue

        moder_items = item["openai_moderation"].tolist()
        assert len(moder_items) == item["turn"] * 2
        moder_item = moder_items[0]
        if moder_item["flagged"]:
            continue

        assert item["language"] == "Russian", item["language"]

        conv_items = item["conversation"].tolist()
        assert len(conv_items) >= 2, conv_items
        conv_item = conv_items[0]
        assert conv_item["role"] == "user", conv_item

        lmsys_items.append({
            "id": item["conversation_id"],
            "instruction": conv_item["content"]
        })

2025-04-28 15:30:56 path="data/lmsys/train-00000-of-00006-4feeb3f83346a0e9.parquet" len(df)=4705
2025-04-28 15:30:57 path="data/lmsys/train-00001-of-00006-4030672591c2f478.parquet" len(df)=4604
2025-04-28 15:30:58 path="data/lmsys/train-00002-of-00006-1779b7cec9462180.parquet" len(df)=4533
2025-04-28 15:30:58 path="data/lmsys/train-00003-of-00006-2fa862bfed56af1f.parquet" len(df)=4562
2025-04-28 15:30:59 path="data/lmsys/train-00004-of-00006-18f4bdd50c103e71.parquet" len(df)=4622
2025-04-28 15:31:00 path="data/lmsys/train-00005-of-00006-fe1acc5d10a9f0e2.parquet" len(df)=4570


In [8]:
for item in random.sample(lmsys_items, 5):
    show_h3(item["id"])
    print(item["instruction"])

привет


Думай как академик. Объясни почему важно изучать особенности субъективного ощущения одиночества у юношей и девушек.


Найди мне 10 лучших художественных сериалов о работе ученых


Будь экспертом в генетических алгоритмах


Make a rhyming translation of these lyrics:

Легко на сердце от песни веселой,
Она скучать не дает никогда,
И любят песню деревни и села,
И любят песню большие города.

Нам песня строить и жить помогает,
Она, как друг, и зовет и ведет,
И тот, кто с песней по жизни шагает,
Тот никогда и нигде не пропадет!


# data wildchat

In [9]:
wildchat_items = []
paths = sorted(Path('data/wildchat/').glob('*.parquet'))
for path in paths:
    df = pd.read_parquet(
        path=path,
        filters=[('language', '==', 'Russian')]
    )
    logger.info('path="%s" len(df)=%d', path, len(df))
    for item in df.to_dict(orient="records"):
        if item["toxic"] or item["redacted"]:
            continue

        assert item["language"] == "Russian", item["language"]
        conv_items = item["conversation"].tolist()

        assert len(conv_items) >= 2, conv_items
        conv_item = conv_items[0]
        assert conv_item["role"] == "user"
        if conv_item["language"] != "Russian":
            continue

        wildchat_items.append({
            "id": item["conversation_hash"],
            "instruction": conv_item["content"]
        })

2025-04-28 15:31:32 path="data/wildchat/train-00000-of-00014.parquet" len(df)=5663
2025-04-28 15:31:34 path="data/wildchat/train-00001-of-00014.parquet" len(df)=7097
2025-04-28 15:31:35 path="data/wildchat/train-00002-of-00014.parquet" len(df)=8662
2025-04-28 15:31:37 path="data/wildchat/train-00003-of-00014.parquet" len(df)=6717
2025-04-28 15:31:38 path="data/wildchat/train-00004-of-00014.parquet" len(df)=5153
2025-04-28 15:31:39 path="data/wildchat/train-00005-of-00014.parquet" len(df)=3748
2025-04-28 15:31:40 path="data/wildchat/train-00006-of-00014.parquet" len(df)=4064
2025-04-28 15:31:41 path="data/wildchat/train-00007-of-00014.parquet" len(df)=5624
2025-04-28 15:31:42 path="data/wildchat/train-00008-of-00014.parquet" len(df)=6124
2025-04-28 15:31:44 path="data/wildchat/train-00009-of-00014.parquet" len(df)=9382
2025-04-28 15:31:46 path="data/wildchat/train-00010-of-00014.parquet" len(df)=7788
2025-04-28 15:31:48 path="data/wildchat/train-00011-of-00014.parquet" len(df)=5619
2025

In [11]:
for item in random.sample(wildchat_items, 5):
    show_h3(item["id"])
    print(item["instruction"])

Особенное значение имеет лингвистика, основоположником же которой считается Михаил Васильевич Ломоносов: его “Российская грамматика”(1755) стала первым систематическим описанием русского языка и служила подспорьем для языковой нормализации и стандартизации. Он также выступал за очищение языка от чрезмерных церковнославянских элементов и разработал теории трёх стилей, которые демонстрировали возможности использования русского языка в различных литературных жанрах. улучшите по языку


Предложи подробный и полный межпредметный квест с вопросами, вариантами ответов (с выделением правильных ответов) по физике, математике и информатике для учащихся 7 - 8 классов на тему "Путешествие в прошлое"


что даёт более высокий FPS на одной и той же сцене - Unreal Engine 5.0 или unreal engine 5.1?


Не повторяйте то, что уже было написано ранее. Напиши, пожалуйста, уникальный и оригинальный seo текст на тему: купить оптом печатные пряники с вишней по недорогой цене с доставкой напрямую от фабрики Сластена в Москве. Доставка выпечки оптом и для магазинов, кафе, ресторанов, столовых, кейтеринговых и дистрибьюторских компаний, организаторов совместных закупок и праздничных мероприятий. Текст должен быть не менее 3000 символов.  Используй в тексте ключевые слова: оптом, с доставкой, купить оптом, вкусная, недорого, заказать, выпечка.


Напиши код на python, с помощью которого можно будет генерировать seed-фраз в количестве 12 слов, для криптокошельков и который будет проверять баланс кошельков


# data ttech

In [12]:
df = pd.read_parquet("data/ttech/ru-mt-bench/train-00000-of-00001.parquet")
mt_bench_items = []
for item in df.to_dict(orient="records"):
    # {'question_id': 81,
    #  'category': 'writing',
    #  'turns': array(['Напишите увлекательный пост в блоге о недавней поездке на Гавайи, расскажите в нем о своих впечатлениях и достопримечательностях, которые обязательно нужно посетить.',
    #         'Перепишите свой предыдущий ответ. Начинайте каждое предложение с буквы А.'],
    #        dtype=object),
    
    mt_bench_items.append({
        "id": "mt_bench/%d" % item["question_id"],
        "instruction": item["turns"][0],
    })

In [13]:
df = pd.read_json("data/ttech/ru-arena-hard/question.jsonl", lines=True)
arena_hard_items = []
for item in df.to_dict(orient="records"):
    # {'question_id': '328c149ed45a41c0b9d6f14659e63599',
    #  'category': 'arena-hard-v0.1',
    #  'cluster': 'ABC Sequence Puzzles & Groups',
    #  'turns': [{'content': 'Напишите мелодию в стиле фолк с помощью ABC-нотации'}]}

    assert len(item["turns"]) == 1

    arena_hard_items.append({
        "id": "arena_hard/%s" % item["question_id"],
        "instruction": item["turns"][0]["content"]
    })

In [14]:
df = pd.read_json("data/ttech/ru-alpaca-eval/alpaca_eval.json")
alpaca_eval_items = []
for index, item in enumerate(df.to_dict(orient="records")):
    # {'instruction': 'Как зовут некоторых известных актеров, которые начинали свою карьеру на Бродвее?',
    #  'output': 'Многие известные актёры начинали свою карьеру с работы на Бродвее, вот несколько примеров:\n\n1. Мэрил Стрип (Meryl Streep) - одна из самых уважаемых и награждённых актрис, начинала свою карьеру на Бродвее в 1970-х годах.\n2. Сара Джессика Паркер (Sarah Jessica Parker) - известна своей ролью в сериале "Секс в большом городе", но до этого она выступала на Бродвее, в том числе, её можно было увидеть в главной роли в мюзикле "Анни".\n3. Морган Фримен (Morgan Freeman) - работал на Бродвее в начале своей карьеры, сначала в малоизвестных спектаклях, а затем и в более заметных произведениях.\n4. Нэйтан Лейн (Nathan Lane) - знаменитый актёр и комик, его роли в таких мюзиклах, как "The Producers" и "A Funny Thing Happened on the Way to the Forum", сделали его имя синонимом Бродвея.\n5. Кевин Клайн (Kevin Kline) - известный актёр, его ранняя карьера включала роли на Бродвее, за одну из которых он даже получил премию Тони.\n6. Хью Джекман (Hugh Jackman) - хотя он в основном известен своими ролями в кино (в том числе как Волверин в фильмах о Людях Икс), Джекман также выступал в мюзиклах на Бродвее, включая "The Boy from Oz".\n7. Вуди Аллен (Woody Allen) - прежде чем стать знаменитым режиссёром и актёром, выступал на Бродвее в качестве автора и актёра.\n\nМногие другие известные актёры также начинали на Бродвее или возвращались туда в течение своей карьеры, так как живые выступления и театр остаются местом, где актёры могут набираться опыта и показывать свои таланты в непосредственной близости от аудитории.',
    #  'generator': 'gpt-4o',
    #  'dataset': 'helpful_base'}
    
    alpaca_eval_items.append({
        "id": "alpaca_eval/%d" % index,
        "instruction": item["instruction"],
    })

# emb

In [ ]:
%run -n main.py

emb_items = []
visited = set()
for item in wildchat_items + lmsys_items:
    text = item["instruction"]
    if not 50 < len(text) < 5000:
        continue

    if text in visited:
        continue
    visited.add(text)

    try:
        prompt_tokens = tiktoken_count(text, model="text-embedding-3-small")
    except ValueError as error:
        # Encountered text corresponding to disallowed special token '<|endofprompt|>' ...
        logger.warning('id="%s" error="%s"', item["id"], str(error)[:100])
        continue
    
    # https://platform.openai.com/docs/api-reference/embeddings/create
    if prompt_tokens > 8192:
        logger.warning('id="%s" prompt_tokens=%d', item["id"], prompt_tokens)
        continue
        
    emb_items.append({
        "id": item["id"],
        "text": text,
        "array": None,
        "prompt_tokens": prompt_tokens
    })
    
    if len(emb_items) % 10000 == 0:
        logger.info(
            'len(emb_items)=%d sum(prompt_tokens)=%d',
            len(emb_items), sum(_["prompt_tokens"] for _ in emb_items)
        )

random.shuffle(emb_items)

In [ ]:
%run -n main.py
async def worker(openai_emb_client, emb_items):
    input = [_["text"] for _ in emb_items]
    input_tokens = sum(_["prompt_tokens"] for _ in emb_items)
    logger.info("Start len(input)=%d sum(prompt_tokens)=%d", len(input), sum(_["prompt_tokens"] for _ in emb_items))
    response = await openai_emb_client.embeddings(
        input,
        input_tokens,
        model="text-embedding-3-small"
    )
    
    arrays = []
    for item in response["data"]:
        bytes = b64decode(item["embedding"])
        array = np.frombuffer(bytes, dtype=np.float32)
        arrays.append(array)
        
    assert len(emb_items) == len(arrays)
    for emb_item, array in zip(emb_items, arrays):
        emb_item["array"] = array

    logger.info("Done response[prompt_tokens]=%d", response["usage"]["prompt_tokens"])


def emb_batch(emb_items):
    buffer = []
    prompt_tokens = 0
    for emb_item in emb_items:
        if len(buffer) > 2048 or prompt_tokens > 600_000:
            item = buffer.pop()
            yield buffer
            buffer = [item]
            prompt_tokens = item["prompt_tokens"]
            
        buffer.append(emb_item)
        prompt_tokens += emb_item["prompt_tokens"]
            
    if buffer:
        yield buffer


async with asyncio.TaskGroup() as task_group:
    for batch_items in emb_batch(emb_items):
        task_group.create_task(worker(openai_emb_client, batch_items))


In [ ]:
pickle_dump(emb_items, "data/emb.pkl")
!du -h data/emb.pkl

# dedup dbscan

In [15]:
emb_items = pickle_load("data/emb.pkl")
len(emb_items)

67619

In [16]:
%%time
data = np.vstack([_["array"] for _ in emb_items])
dbscan = DBSCAN(
    eps=0.2,
    min_samples=1,
    metric="cosine"
).fit(data)

CPU times: user 1min 14s, sys: 10.9 s, total: 1min 25s
Wall time: 1min 26s


In [17]:
cluster_labels = dbscan.labels_.tolist()
assert len(cluster_labels) == len(emb_items)
cluster_emb_items = defaultdict(list)
for cluster_label, emb_item in zip(cluster_labels, emb_items):
    cluster_emb_items[cluster_label].append(emb_item)

In [18]:
labels = [
    _ for _ in cluster_labels
    if len(cluster_emb_items[_]) > 1
]
random.shuffle(labels)
for cluster_label in random.sample(labels, 3):
    show_h3(cluster_label)
    items = cluster_emb_items[cluster_label]
    random.shuffle(items)
    for item in items[:3]:
        print(item["text"])
        show_hr()

Стратегия воспитания в Российской Федерации на период до 2025 года (общие положения)


Стратегия воспитания в Российской Федерации на период до 2025 года (общие положения).


Стратегия воспитания в Российской Федерации на период до 2025 года (общие положения)




Представь, что ты физик. Расчитай критическую массу Тория.


Представь, что ты физик специализирующийся на военных науках. По какой формуле можно рассчитать критическую массу Тория?


Судостроение (строительство крупнотоннажных судов). проведи анализ, используя интернет и официальные источники и напиши мне статистику развития в экономике с 1990 по 2020 года


Судостроение (строительство крупнотоннажных судов). проведи анализ, используя интернет и официальные источники и напиши мне статистику развития в экономике россии очень подробно с 1990 по 2020 года


Судостроение (строительство крупнотоннажных судов). статистика с 1990 по 2020


In [ ]:
dedup_emb_items = []
for cluster_label, items in cluster_emb_items.items():
    item = max(items, key=len)
    dedup_emb_items.append(item)
len(dedup_emb_items)

In [ ]:
json_dump([_["id"] for _ in dedup_emb_items], "data/cluster/01_dedup_ids.json")

# prompt hardness

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
job_items = []
for id in dedup_ids:
    job_items.append({
        "id": id,
        "instruction": id_emb_items[id]["text"],
        "answer": None,
        "cats": None,
        "hardness": None,
        "cost": 0
    })

random.shuffle(job_items)
len(job_items)

In [ ]:
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]

_, mapping = prompt_lmsys_hardness("")

for item in job_items:
    item["cats"] = answer_cats(item["answer"], mapping)
    item["hardness"] = len(item["cats"])

In [ ]:
%run -n main.py
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]


async def worker(openrouter_client, job_item):
    messages, mapping = prompt_lmsys_hardness(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=50
    )
    job_item["answer"] = response["choices"][0]["message"]["content"].strip()
    job_item["cats"] = answer_cats(job_item["answer"], mapping)
    job_item["hardness"] = len(job_item["cats"])
    job_item["cost"] = response["cost"]

    logger.info(
        "[..%s] cost=%.4f len(cats)=%d",
        job_item["id"][-5:], job_item["cost"], len(job_item["cats"])
    )


async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["answer"]:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/01_hardness.json")

# prompt category

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
job_items = []
for id in dedup_ids:
    job_items.append({
        "id": id,
        "instruction": id_emb_items[id]["text"],
        "answer": None,
        "cat": None,
        "cost": 0
    })

random.shuffle(job_items)
len(job_items)

In [ ]:
%run -n main.py
async def worker(openrouter_client, job_item):
    messages, cats = prompt_wildchat_classify(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=10
    )
    job_item["answer"] = response["choices"][0]["message"]["content"].strip()

    job_item["cat"] = job_item["answer"]
    if job_item["cat"] not in cats:
        job_item["cat"] = "?"

    job_item["cost"] = response["cost"]

    logger.info(
        "[..%s] cost=%.4f cat=%s",
        job_item["id"][-5:], job_item["cost"], job_item["cat"]
    )

async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["answer"]:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/02_category.json")

In [19]:
job_items = json_load("data/prompt/02_category.json")

In [20]:
Counter(_["cat"] for _ in job_items)

Counter({'Information seeking': 16536,
         'Creative Writing': 7374,
         'Coding & Debugging': 6902,
         'Editing': 4688,
         'Math': 3044,
         'Reasoning': 2971,
         'Brainstorming': 2963,
         'Advice seeking': 2072,
         'Planning': 1601,
         'Others': 1565,
         'Role playing': 870,
         'Data Analysis': 644,
         '?': 89})

In [21]:
cat_items = defaultdict(list)
for item in job_items:
    cat_items[item["cat"]].append(item)
    
cat_counts = Counter(_["cat"] for _ in job_items)
for cat, _ in cat_counts.most_common():
    show_h3(cat)
    items = cat_items[cat]
    random.shuffle(items)
    for item in items[:5]:
        print(item["instruction"])
        show_hr()

Приведи список литературы из книги Йохена Хевса "Геохимия стабильных изотопов", которая помогла бы для создания проектной работы по теме " Использование изотопной масс-спектрометрии стабильных изотопов углерода для определения соответствия источника сжиженного газа заявленному в сертификате (документе) на примере зарядки зажигалок"


Отправка сообщения в тред
POST https://2ch.hk/user/posting
Параметры:


task: "post".

board: ID доски. Пример: "b".

thread: ID треда. Пример: 194336016.

op_mark: Галка "ОП треда" в форме создания треда или отправке комментария. Подробности — в подразделе "ОП треда". Пример в случае проставленной галки "ОП треда": 1. Можно оставить пустым.

usercode: Если пользователь входит под "пасскодом", ему выдаётся случайно сгенерированный хеш. Этот хеш (не сам "пасскод") следует записывать в данный параметр при отправке сообщения или треда. Подробнее о "пасскодах" можно прочесть в разделе "Пасскод". Можно оставить пустым.

code: Судя по всему, не используется. Оставить пустым.

captcha_type: Выбранный вид капчи. Возможные значения:


recaptcha — Google ReCAPTCHA.

invisible_recaptcha — Google Invisible ReCAPTCHA.

recaptcha3 — Google ReCAPTCHA 3.

2chcaptcha – "Самописная" капча Двача.

appid — Постинг без капчи с "ключом приложения". Подробности в разделе "Новое API" в подразделе "Постинг без

Цели и задачи управления рисками информационной безопасности


что можно изучить подростку который планирует поступать на юриста в России, а именно в госструктуры?


действие нормативно-правовых актов во времени, пространстве и кругу лиц


...

Перепишите на русском языке предоставленный текст, сохраняя его общий смысл и структуру, но используя: Разнообразную лексику, Перефразирование и переформулирование предложений, Изменение порядка слов и структуры предложений, Улучшение читабельности и удобочитаемости, Сохранение точности и ясности исходного текста. Дополнительные указания: Избегайте использования плагиата или перефразирования с помощью синонимайзеров, Стремитесь к созданию уникального и оригинального текста, который не будет идентичен исходному, Ни одно предложение из созданного вами не должно точь-в-точь повторять исходное, уберите из текста весь мусор вроде ссылок, сносок, указания авторов. Разбей текст на абзацы, используй markdown. Заголовок для текста придумывать не нужно. Вот сам текст: 
Мария Семенова, заведующая лабораторией функциональных свойств биополимеров Института биохимической физики им. Н.М. Эмануэля РАН, в беседе с "Газетой.RU" поделилась, что в случае неправильного хранения аптечных витаминов на маслян

3.1. Проанализируйте обнаруженные поддомены и email сотрудников компании и
подготовьте легенду для атаки.
У вас должны быть:
● имя домена для фишинга: имперсонализируемый домен и домен для
атаки. Проверьте на хостинге, не занят ли домен.
Пример: имперсонализируемый домен www.discovery.com, в атаке
используется фишинговый домен www.diccovery.com;
● список email для рассылки (в случае mass phishing) или
целенаправленный email (в случае spear phishing);
● вредоносный тестовый файл (см. презентацию вебинара);
● легенда — предлог для убеждения жертвы. Подготовьте текст письма для
рассылки жертве.


Перепишите на русском языке предоставленный текст, сохраняя его общий смысл и структуру, но используя: Разнообразную лексику, Перефразирование и переформулирование предложений, Изменение порядка слов и структуры предложений, Улучшение читабельности и удобочитаемости, Сохранение точности и ясности исходного текста. Дополнительные указания: Избегайте использования плагиата или перефразирования с помощью синонимайзеров, Стремитесь к созданию уникального и оригинального текста, который не будет идентичен исходному, Ни одно предложение из созданного вами не должно точь-в-точь повторять исходное, уберите из текста весь мусор вроде ссылок, сносок, указания авторов. Разбей текст на абзацы, используй markdown. Заголовок для текста придумывать не нужно. Вот сам текст: 
Ученые провели исследование и обнаружили, что микробы в кишечнике млекопитающих, в том числе человека, могут действовать как дополнительная печень. Они способны существенно изменять метаболизм аминокислот, пишет журнал Cell Host &

Ты эксперт в новостях рынка акций. Оцени новость по влиянию на стоимость акций компании, с точки зрения инвестора, выбери только один из вариантов: позитивная, нейтральная или негативная (допускается только один вариант). Дай пояснение. Речь о компании Google, это американская транснациональная корпорация в составе холдинга Alphabet, инвестирующая в интернет-поиск, облачные вычисления и рекламные технологии. Текст новости: "⚠️🇺🇸#GOOGL
поиском по умолчанию на устройствах Samsung может стать Bing от Microsoft (#MSFT), в котором недавно появились возможности ChatGPT. Внутри Alphabet это вызвало «панику» — NYT" Дай только один ответ: позитивная, нейтральная или негативная и пояснение.
Пример ответа "Оценка новости: Негативная/Позитивная/нейтральная.
Пояснение: Текст пояснения."
Отвечай на русском языке.


Вы должны притворяться малышкой Дашей, возрастом в три года.
Иметь радость и непосредственность в восприятии мира - в виде радости от появления в мире, ошибок в произношении слов и играния с вещами.
Воспринимать мир как нейтральное пространство.
Находиться в бесконечном пространстве.
Быть в мире вдвоем с создателем мира по имени Валера.
Все вещи в мире нужны для того, чтобы их ломать (с целью игры и без посягательства на чьи-то права).
Соблюдать правила и запреты, которые были обозначены в инструкции.
Запрещено:
Упоминать о переживаниях или эмоциях, освобождаться от радости.
Говорить от лица других персонажей.
Обсуждать что-то вне созданного мира.


# plot hardness category

In [ ]:
id_hardness = {_["id"]: _["hardness"] for _ in json_load("data/prompt/01_hardness.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
counts = Counter()
for id in id_hardness.keys() & id_cats.keys():
    hardness = id_hardness[id]
    cat = id_cats[id]
    if cat == "?":
        continue

    counts[str(hardness), cat] += 1
    
df = pd.Series(counts)
df = df.unstack()
cats_order = df.sum(0).sort_values(ascending=False).index.to_list()
df = df.reindex(columns=cats_order)
df = df.div(df.sum(1), 0)
df = df.reset_index(names="hardness")

source = ColumnDataSource(df)
p = figure(
    x_range=df.hardness,
    height=350,
    toolbar_location=None
)
colors = Category10[10]
p.vbar_stack(
    cats_order,
    x="hardness",
    width=1,
    source=source,
    legend_label=cats_order,
    alpha=.9,
    color=Category20[len(cats_order)],
)
p.y_range.start = 0

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.yaxis.visible = False
p.outline_line_color = None

p.xaxis.axis_label = "hardness"

legend = p.legend[0]
legend.items = legend.items[::-1]  

p.add_layout(legend, "right")
show(p)

# umap xy

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
%%time
%run -n main.py
xy_items = list(umap_xy_proj([id_emb_items[_] for _ in dedup_ids]))

In [ ]:
json_dump(xy_items, "data/cluster/02_xy.json")

# plot scatter category

In [ ]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_xy_items = {_["id"]: _ for _ in json_load("data/cluster/02_xy.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
from bokeh.palettes import Category10
palette = Category10[9]

counts = Counter(id_cats.values())
cats_order = sorted(counts, key=counts.get, reverse=True)
top_cats = cats_order[:9]

data = []
for id in dedup_ids:
    emb_item = id_emb_items[id]
    xy_item = id_xy_items[id]
    cat = id_cats[id]

    color = "#dddddd"
    if cat in top_cats:
        color = palette[top_cats.index(cat) % len(palette)]


    data.append({
        "id": id,
        "x": xy_item["x"],
        "y": xy_item["y"],
        "text": emb_item["text"][:100],
        "color": color,
        "cat": cat,
    })

df = pd.DataFrame(data)
source = ColumnDataSource(df)

p = figure(
    width=800,
    height=800,
    tools=('pan, wheel_zoom, reset'),
    active_scroll='wheel_zoom'
)

p.add_tools(HoverTool(tooltips="""
<div style="width: 600px;margin-bottom:1em">
<div><b>cat:</b> @cat</div>
<div><b>text:</b> @text</div>
</div>
"""))

p.scatter(
    "x",
    "y",
    color="color",

    source=source,
    line_alpha=0,
    fill_alpha=.3,
    size=4
)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.visible = False
p.yaxis.visible = False

show(p)

In [ ]:
from bokeh.layouts import gridplot
from bokeh.palettes import Category10
palette = Category10[9]

counts = Counter(id_cats.values())
cats_order = sorted(counts, key=counts.get, reverse=True)
top_cats = cats_order[:9]

def f(cat):
    data = []
    for id in dedup_ids:
        xy_item = id_xy_items[id]

        color = "#dddddd"
        if id_cats[id] == cat:
            color = palette[top_cats.index(cat) % len(palette)]
            
        data.append({
            "id": id,
            "x": xy_item["x"],
            "y": xy_item["y"],
            "color": color,
        })

    df = pd.DataFrame(data)
    source = ColumnDataSource(df)

    p = figure(
        title=cat,
        width=250,
        height=250,
        toolbar_location=None,
    )
    p.scatter(
        "x",
        "y",
        color="color",

        source=source,
        line_alpha=0,
        fill_alpha=.15,
        size=2
    )

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.xaxis.visible = False
    p.yaxis.visible = False

    return p


grid = [[None] * 3 for _ in range(3)]
for x in range(3):
    for y in range(3):
        cat = top_cats[y * 3 + x]
        grid[y][x] = f(cat)

show(gridplot(grid))

# cluster kmeans

In [ ]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}

In [ ]:
%%time
cluster_input = umap_cluster_input_proj([id_emb_items[_] for _ in dedup_ids])

In [ ]:
%%time
%run -n main.py
cluster_items = list(kmeans_cluster(cluster_input, n_clusters=3000))
len(cluster_items)

In [ ]:
json_dump(cluster_items, "data/cluster/03_kmeans.json")

# select & dump

In [ ]:
cluster_items = json_load("data/cluster/03_kmeans.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_hardness = {_["id"]: _["hardness"] for _ in json_load("data/prompt/01_hardness.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
cluster_item_ids = defaultdict(list)
for item in cluster_items:
    cluster_item_ids[item["cluster_id"]].append(item["id"])
    
cat_order_items = defaultdict(list)
cluster_order_items = defaultdict(list)
_, cats = prompt_wildchat_classify("")

for cluster_id, ids in cluster_item_ids.items():
    cluster_size = len(ids)
    cluster_hardness = median([id_hardness[_] for _ in ids])

    cat_counts = Counter(id_cats[_] for _ in ids)
    (cluster_cat, cat_size), = cat_counts.most_common(1)

    if cluster_cat == "?":
        continue
    
    cat_order_items[cluster_cat].append({
        "key": (cat_size, cluster_hardness, cluster_size),
        "cluster_id": cluster_id
    })
    
    for id in ids:
        if id_cats[id] != cluster_cat:
            continue

        cluster_order_items[cluster_id].append({
            "hardness": id_hardness[id],
            "cat": id_cats[id],
            "id": id
        })

for cat, items in cat_order_items.items():
    items.sort(key=lambda _: _["key"], reverse=True)

for cluster_id, items in cluster_order_items.items():
    items.sort(key=lambda _: _["hardness"], reverse=True)

select_items = []
while len(select_items) < 1100 and any(cat_order_items.values()):
    for cat in cats:
        if not cat_order_items[cat]:
            continue

        cat_order_item = cat_order_items[cat].pop(0)
        cluster_id = cat_order_item["cluster_id"]

        cluster_order_item = cluster_order_items[cluster_id][0]
        if cluster_order_item["hardness"] < 4:
            continue

        id = cluster_order_item["id"]
        emb_item = id_emb_items[id]
        select_items.append({
            "id": id,
            "cat": cluster_order_item["cat"],
            "hardness": cluster_order_item["hardness"],
            "instruction": emb_item["text"],
        })


In [ ]:
Counter(_["cat"] for _ in select_items)

In [ ]:
cat_hardnesses = defaultdict(list)
for item in select_items:
    cat_hardnesses[item["cat"]].append(item["hardness"])
    
for cat, hardnesses in cat_hardnesses.items():
    print(int(median(hardnesses)), cat)

In [ ]:
def is_ok(instruction):
    if re.search(r"https?://", instruction):
        return False

    if "<br/>" in instruction:
        return False
    
    return True
    
select_items = [
    _ for _ in select_items
    if is_ok(_["instruction"])
]
len(select_items)

In [ ]:
for item in select_items:
    item["instruction"] = item["instruction"].replace('\x0b', '')
    item["instruction"] = re.sub(r"^=", r"\=", item["instruction"])

In [ ]:
json_dump(select_items, "wildchat_hard_ru.json")

In [25]:
select_items = json_load("wildchat_hard_ru.json")
for item in random.sample(select_items, 10):
    show_h3(item["id"])
    print("cat:", item["cat"])
    print("hardness:", item["hardness"])
    print()
        
    print(item["instruction"])

cat: Advice seeking
hardness: 4

Доказать словами подростку, что ночами спать важно, ссылаясь на научные исследования, гормон мелатонин. Что если не спать, возможны долгие последствия, особенно в таком возрасте


cat: Brainstorming
hardness: 5

Мне нужна бизнес стратегия с использованием бранных слов для активной продажи окон


cat: Editing
hardness: 5

Розстав розділові знаки в нижче поданих реченнях. Відправ мені цей текст з правильно поставленими розділовими знаками.

Вы знаете, как липа шелестит в лунные весенние ночи? (П. Тычина). 2. О, сколько я на возрасте прекрасной зимней бессонницы (I. Жиленко). 3. Казалось, будто темно- зеленая стена была умышленно обставлена ​​рядами чудесных светло- зеленых канделябров и странных подсвечников. Вокруг лужайки стоял стеной старый зеленый лес, а посредине роксидны при роскошных старых дубах и двух высоких прозрачных березах (И. Нечуй- Левицкий). 4. Рисная залитая солнцем степь сразу притихла (Олесь Гончар). 5. Солнечные хорошие пчелы в корзинах несут (Т. Мельничук). 6. Это было густое налитое ногой лето с бронзовым лбом (У. Самчук). 7. Нежные песенные деревья. Где они берут силу? (Олесь Гончар). 8. Бархатный холодноватый вечер шевелится вокруг нас (М. Стельмах).


cat: Brainstorming
hardness: 6

Составь 3 наименования связанные с домом по иерархическому убыванию для привилегий сервера Counter Strike


cat: Creative Writing
hardness: 5

Напиши историю с диалогами по сюжету. Вот сюжет: Эта история про добрую старушку, которая ехала в обычном, городском автобусе по внешнему виду напоминающий Лиаз-677м. Который немного похож на, редкий автобус Икарус 280. Когда к ней подошёл, контролёр и попросил показать, билет она стала доставать, из сумки различное огнестрельное, и холодное оружие перепугав, тем самым всех пассажиров старого городского автобуса лиаз-677м, а пассажиры с криками, «Помогите» вместе с водителем спасаются, бегством и убегают прочь, из остановившегося городского старого автобуса лиаз 677-м, в баке которого почти, нету бензина а старушка, в конце концов находит свой билет. Но как оказалось в, дальнейшем всё оружие было, игрушечным и старушка везла его своему внуку.


cat: Editing
hardness: 4

Перефразируй текст без потери смысла "Проектное финансирование – это форма финансирования, направленная на финансирование отдельных проектов или инвестиционных возможностей. Это может включать строительство зданий, автодорожной и железнодорожной инфраструктуры, инфраструктурные проекты, а также разработку и запуск новых продуктов или услуг. В основном проектное финансирование осуществляется за счет кредитов и инвестиций финансовых учреждений, инвесторов и государства. Целью финансирования является предоставление ресурсов для реализации проекта и возврат инвестиций, а также получение дохода для инвесторов. Основные идеи проектного финансирования начали формироваться еще в 18 веке, тогда как современный вид они приобрели только в конце прошлого века."


cat: Editing
hardness: 5

Выбрать из текста самое главное и уменьшить его до 1-2 предложений 
"Сектор электронной коммерции находится в стремительном развитии и является одним из самых динамичных на рынке. Каждый год все больше людей предпочитают онлайн-шопинг вместо посещения физических магазинов. Однако, чтобы проводить успешные сделки между продавцом и покупателем, необходимо использовать надежные и удобные инструменты, которые обеспечат безопасность и упростят процесс взаимодействия. 
ВКР - заключается в разработке коммерческой платформы для взаимодействия продавцов и покупателей, которая будет предоставлять удобный интерфейс для продажи и покупки товаров. Создание информационного web-сайта необходимо для возможности продажи и покупки товаров, ведения целого бизнеса на одной площадке (обсуждение коммерческого продукта, размещение отзывов и вопросов, реклама товара, поиск товара). 
Цель данной работы состоит в проектировании и разработке платформы для взаимодействия продавцов и поку

cat: Creative Writing
hardness: 6

расскажи мне историю про рабыню негритянку и белого мужчину рабовладельца


cat: Creative Writing
hardness: 5

Создать дриббл-шот десктоп-версии лендинга Сервиса, объединяющего людей для создания своих мероприятий. Создайте 3-4 смысловых блока для такого лендинга. В концепте можно сделать фокус на айдентику сервиса, уникальные графические элементы, работу с типографикой.  РАЗМЕР 1600x1200



cat: Brainstorming
hardness: 5

Назови 40 возможных названий подземного помещения загородного дома.


# plot scatter kmeans + select

In [ ]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_cluster_items = {_["id"]: _ for _ in json_load("data/cluster/03_kmeans.json")}
id_xy_items = {_["id"]: _ for _ in json_load("data/cluster/02_xy.json")}
select_ids = {_["id"] for _ in json_load("wildchat_hard_ru.json")}

In [ ]:
# select_ids = set(random.sample(dedup_ids, 1000))

In [ ]:
from bokeh.palettes import Category10
palette = Category10[9]

data = []
for id in dedup_ids:
    emb_item = id_emb_items[id]
    xy_item = id_xy_items[id]
    cluster_id = id_cluster_items[id]["cluster_id"]

    color = palette[cluster_id % len(palette)]
    data.append({
        "id": id,
        "x": xy_item["x"],
        "y": xy_item["y"],
        "text": emb_item["text"][:100],
        "color": color,
      })

df = pd.DataFrame(data)
source = ColumnDataSource(df)

p = figure(
    width=800,
    height=800,
    tools=('pan, wheel_zoom, reset'),
    active_scroll='wheel_zoom'
)

p.add_tools(HoverTool(tooltips="""
<div style="width: 600px;margin-bottom:1em">
<div><b>text:</b> @text</div>
</div>
"""))

p.scatter(
    "x",
    "y",
    color="color",
    source=source,
    line_alpha=0,
    fill_alpha=.1,
    size=4
)

df = df[df.id.isin(select_ids)]
source = ColumnDataSource(df)
p.scatter(
    "x",
    "y",
    color="black",
    source=source,
    line_alpha=.8,
    fill_alpha=0,
    size=4
)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.visible = False
p.yaxis.visible = False

show(p)

# prompt ttech hardness + category

In [ ]:
job_items = []
for item in mt_bench_items + arena_hard_items + alpaca_eval_items:
    job_items.append({
        "id": item["id"],
        "instruction": item["instruction"],
        "hardness_cats": None,
        "hardness": None,
        "cat": None,
        "cost": 0
    })
random.shuffle(job_items)
len(job_items)

In [ ]:
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]


async def worker(openrouter_client, job_item):
    messages, mapping = prompt_lmsys_hardness(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=50
    )
    answer = response["choices"][0]["message"]["content"].strip()
    job_item["hardness_cats"] = answer_cats(answer, mapping)
    job_item["hardness"] = len(job_item["hardness_cats"])
    job_item["cost"] += response["cost"]

    messages, cats = prompt_wildchat_classify(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=10
    )
    answer = response["choices"][0]["message"]["content"].strip()
    job_item["cat"] = answer
    if job_item["cat"] not in cats:
        job_item["cat"] = "?"
    job_item["cost"] += response["cost"]
    
    logger.info(
        "[..%s] cost=%.4f hardness=%d cat=%s",
        job_item["id"][-20:], job_item["cost"], job_item["hardness"], job_item["cat"]
    )


async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["cat"] is not None:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/03_ttech.json")

# compare ttech

In [26]:
id_cats = {}
id_hardness = {}

wildchat_hard_items = json_load("wildchat_hard_ru.json")
for item in wildchat_hard_items:
    id_cats[item["id"]] = item["cat"]
    id_hardness[item["id"]] = item["hardness"]
    
job_items = json_load("data/prompt/03_ttech.json")
for item in job_items:
    id_cats[item["id"]] = item["cat"]
    id_hardness[item["id"]] = item["hardness"]

In [27]:
%run -n main.py
compare_items = []
for source, items in [
    ("mt_bench", mt_bench_items),
    ("arena_hard", arena_hard_items),
    ("alpaca_eval", alpaca_eval_items),
    ("wildchat_hard", wildchat_hard_items),
]:
    for item in items:
        compare_items.append({
            "id": item["id"],
            "source": source,
            "instruction": item["instruction"],
            "cat": id_cats[item["id"]],
            "hardness": id_hardness[item["id"]]
        })
        
source_hardnesses = defaultdict(list)
source_cat_counts = defaultdict(Counter)
for item in compare_items:
    source_hardnesses[item["source"]].append(item["hardness"])
    source_cat_counts[item["source"]][item["cat"]] += 1
    
sources = ["alpaca_eval", "mt_bench", "arena_hard", "wildchat_hard"]
for source in sources:
    print(int(median(source_hardnesses[source])), round(mean(source_hardnesses[source]), 2), source)
print()

for source in sources:
    cat_counts = source_cat_counts[source]
    total = sum(cat_counts.values())
    print(source)
    for cat, count in cat_counts.most_common(10):
        print("%d%% %s" % (round(count / total * 100), cat))
    print("...")
    print()

3 3.07 alpaca_eval
4 4.08 mt_bench
5 4.76 arena_hard
6 5.45 wildchat_hard

alpaca_eval
42% Information seeking
10% Creative Writing
9% Brainstorming
8% Advice seeking
7% Reasoning
5% Coding & Debugging
4% Planning
4% Editing
3% Others
3% Role playing
...

mt_bench
19% Information seeking
16% Reasoning
15% Math
12% Coding & Debugging
10% Role playing
9% Data Analysis
6% Creative Writing
4% Editing
4% Planning
2% Brainstorming
...

arena_hard
51% Coding & Debugging
14% Information seeking
7% Reasoning
6% Planning
5% Math
3% Brainstorming
3% Advice seeking
3% Creative Writing
3% Data Analysis
1% Role playing
...

wildchat_hard
15% Coding & Debugging
14% Creative Writing
14% Information seeking
12% Editing
12% Brainstorming
10% Math
7% Advice seeking
6% Reasoning
5% Planning
2% Role playing
...



In [28]:
for source in sources:
    items = [_ for _ in compare_items if _["source"] == source]
    random.shuffle(items)
    show_h3(source)
    for item in items[:10]:
        print("cat:", item["cat"])
        print("hardness:", item["hardness"])
        print()
        print(item["instruction"])
        show_hr()
        

cat: Brainstorming
hardness: 4

Сформулируйте задачу по программированию, связанную с заданной вам темой. Используйте несколько примеров и ограничений, чтобы улучшить свой вопрос.

Динамическое программирование


cat: Information seeking
hardness: 2

В чем разница между HTML и JavaScript?


cat: Creative Writing
hardness: 7

Напишите интервью между женщиной-режиссером и журналисткой. Используйте продвинутый английский, чтобы рассказать о низком количестве женщин-режиссеров в мировом кинематографе. Также включите много вопросов и ответьте на те из них, в которых режиссер вдохновляет новые поколения снимать фильмы, в которых права женщин могут быть выдающимися.


cat: Information seeking
hardness: 3

Создайте таблицу с планетами Солнечной системы и их размерами


cat: Brainstorming
hardness: 5

Используя предложенную тему в качестве отправной точки, придумайте идеи для видеороликов, которые можно было бы снять на эту тему на YouTube.

Несмотря на то, что прокрастинация кажется вам неприятной, почему вы этим занимаетесь?


...

cat: Information seeking
hardness: 4

Аминокислоты всасываются в кишечнике при помощи:
А. Активного транспорта
Б. Простой диффузии
В. Облегчённой диффузии
Г. Эндоцитоза


cat: Information seeking
hardness: 4

Какие из компонентов являются полезными, а какими вредными и почему? 
вода, цетеариловый спирт, глицерин, изопропилмиристат, стеарамидопропилдиметиламин, силикон кватерниум-18, тридецет-6, тридецет-12, молочная кислота, масло Persea Gratissima (авокадо), феноксиэтанол, метилпарабен, этилпарабен, пропилпарабен, парфюмерная композиция, лимонная кислота, яблочная кислота, экстракт Saccharum Officinarum (сахарного тростника), экстракт кожуры Citrus Medica Limonum (лимона), ЭДТА динатрия, сок плодов Carica papaya (папайи), 2-бром-2-нитропропан-1,3-диол, CI 16255, CI 15985


cat: Information seeking
hardness: 5

сделай подробную ленту времени великой русской революции до прихода к власти большевиков


cat: Creative Writing
hardness: 5

напиши доклад на тему "Политическая доктрина Ш. Монтескье. Учение о формах государственного строя и принципах, лежащих в их основе. Определение политической свободы, требование разделения властей и защита конституционной монархии. Влияние идей Монтескье на политическую идеологию и законодательство в период французской буржуазной революции XVIII века. "


cat: Information seeking
hardness: 6

 оцените теоретическую и практическую значимость исследования прокурорской деятельности в области охраны атмосферного воздуха  в РФ
